# Neural Networks in `PyTorch`



We will use a dense neural network in `torch` to solve a simple regression problem using physics data.

## Learning Task
We will construct a dense neural network to predict the invariant mass of a particle from its energy, momentum, (and charge).

*Note that this task does not require machine learning. We choose a task with a known mapping to help us create, debug, and tune our first neural network.*

## Dataset
This dataset is a collection of simulated particle events from [Pythia](https://pythia.org/). The dataset is a 2D array where each row represents one event from an $e^{-} + p$ collision. This dataset is comprised _only_ of events where exactly 16 particles are produced from an electron-proton collision. Each particle contains $(p_x,p_y,p_z,E,q)$ information. Each event is therefore represented by 80 numbers.

**Advanced activity:** There are more interesting event-wise learning tasks using this dataset. Consider crafting your own learning task and target for this data.



## Computational Notes

If this is your first time in a Jupyter-like environment, please read the following carefully:

 - You are in an active kernel
 - Run each cell with `Shift + Enter`
 - You must execute the cells in the order that you want the code to run
 - `Runtime`$\rightarrow$`Change runtime type` allows you to utilize GPUs and TPUs. They are unnecessary here, but will become vital in later exercises.


In [ ]:
# Imports (PyTorch instead of TensorFlow)
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## Data loading

If you already have the data file (e.g. `particle-events.npy`) in your working directory, you can skip this step.



In [ ]:
# import data from github. Note: in colab, go to Files and refresh to see file
# here I use Linux commands within the notebook to pull the data file and rename it
!wget https://github.com/NuclearTalent/MachineLearningECT/blob/master/doc/pub/Day6/data/homogenous-16-particle-events-energy.npy?raw=true
!mv homogenous-16-particle-events-energy.npy?raw=true particle-events.npy

In [ ]:
# now we load the data file, which is a numpy array
events = np.load("particle-events.npy")


Recall that each row of this dataset is an entire event. We need each row to represent a training example, which is a single particle.

Using `numpy`'s `reshape` method we can make each row represent one particle.

At first, we will only use $(p_x,p_y,p_z,E)$ as our features for training a model. We will exclude the charge information. This should be fine, since charge is not needed to compute our target, invariant mass.

We will only use 100 training examples for our first training.

Create a numpy array for this dataset.

In [ ]:
# Here we rearrange the data within each of the events to isolate particles

n_part_per_event = 16      # number of particles per event
n_feat_per_part  = 5       # features per particle (example). If your in

evt_particles = events.reshape( len(events), n_part_per_event, n_feat_per_part)

#print("evt_particles[0] =\n", evt_particles[0])

# Use another call of reshape to combine all events to have the appropriate shape
# Complete me:




# Create a smaller subdataset

# complete me



These are our training data inputs, but we also must provide the targets, which are the invariant masses of each particle. This is a straightforward computation.

We choose units where $c = 1$:
$$m^2=E^2-||\textbf{p}||^2$$
where $m, E$, and $\textbf{p}$ are all in GeV.

**Create an array of your target values, the invariant masses (*not* $m^2$).**
Due to insufficient precision, some $m^2$ values for massless particles will come out very slightly negative. These should be treated as zero to avoid `NaN`. I used the `maximum` method from `numpy` to handle this.


In [ ]:
# Complete me:



Next, make a histogram of the target data to make sure that we are seeing masses of real particles. As this data has limited precision, this will not resolve electrons very well, but protons, pions, and massless particles should be clearly visible.

In [ ]:
plt.hist(m, bins=100)
plt.xlabel("mass (GeV)")
plt.ylabel("count")
plt.title("target distribution")
plt.show()


## Train/validation split and DataLoaders

We will now do a bit more processing to prepare our data for our neural network models. We build our training, validation, and test datasets.

We will first split 20\% of the data for testing, then split 20\% of the remaining data for validation. These are typical splits, but ideal splitting may vary depending on the amount of data you have and presence of outliers.

You likely want to use `scikit-learn`'s [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) method for this task.

In [ ]:
#Note: use scikit-learn instead of this split! Why is this better?
split1 = 5000

X_train = particles[:split1]
y_train = m[:split1].T
#complete me for val and test sets

# Convert to torch tensors
X_tensor_tr = torch.from_numpy(X_train).float()
y_tensor_tr = torch.from_numpy(y_train).float().unsqueeze(1)  # make y shape (N,1)
#complete me for val and test sets


train_ds = torch.utils.data.TensorDataset(X_tensor_tr, y_tensor_tr)
#complete me for val and test sets


# build DataLoader objects
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)


I wrote a simple function for plotting loss curves, which will help us tune our model.

In [ ]:

def plot_learning_curve(history):
    plt.plot(history["loss"], label="training loss")
    plt.plot(history["val_loss"], label="validation loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.yscale("log")
    plt.legend()
    plt.show()

## Now we build our first model architecture!

We will begin with one hidden layer with 2 nodes that have a ReLU activation function. This is **not** an ideal architecture.



In [ ]:
class MLP(nn.Module):
  # Call the parent constructor so nn.Module sets up correctly
  def __init__(self, input_dim):
        super().__init__()
        # nn.Sequential lets us stack layers/modules in order
        # Each layer takes the output of the previous one as input
        self.net = nn.Sequential(
            nn.Linear(input_dim, 5),
            nn.Sigmoid(),
            # complete me!

        )

  def forward(self, x):
     # Defines the forward pass: how input x flows through layers
     return self.net(x)


In [ ]:
model = MLP(input_dim).to(device)
print(model)

# Calculate the number of trainable parameters (check against your own calculation!)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_params)

Defining optimizer, loss, and training loop



In [ ]:
def train_one_epoch(model, loader, optimizer, loss_f):
    # Put the model in training mode
    model.train()
    running_loss = 0.0

    # Loop over all mini-batches from the DataLoader
    for xb, yb in loader:
        # Move both inputs (xb) and targets (yb) onto the same device (CPU or GPU)
        xb, yb = xb.to(device), yb.to(device)
        # Reset gradients from the previous step
        optimizer.zero_grad()
        # Forward pass: compute predictions for this batch
        preds = model(xb)
        # Compute the loss comparing predictions vs. true labels
        loss = loss_f(preds, yb)
         # Backward pass: compute gradients of all model parameters w.r.t. loss
        loss.backward()
        # Take one optimization step (update model weights using gradients)
        optimizer.step()
        # Accumulate the loss, scaled by batch size (for averaging later)
        running_loss += loss.item() * xb.size(0)
    # Return the average loss across the whole dataset
    return running_loss / len(loader.dataset)


# the following is for evaluatinf the model, e.g. for validation
# compare this with the training loop above and try to understand the differences
@torch.no_grad()
def evaluate(model, loader, loss_f):
    model.eval()
    running_loss = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        preds = model(xb)
        loss = loss_f(preds, yb)
        running_loss += loss.item() * xb.size(0)
    return running_loss / len(loader.dataset)

history = {"loss": [], "val_loss": []}





In [ ]:
model = MLP(input_dim).to(device)
print(model)

learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # try stochastic gradient descent
loss_f = nn.MSELoss() # MSE loss function

#training loop!
epochs = 10  # start with 10
for epoch in range(1, epochs+1):
    train_loss = # train one epoch
    val_loss   = # evalaute on val data
    history["loss"].append(train_loss) # keep track of loss
    history["val_loss"].append(val_loss) # keep track of val loss
    if epoch % 1 == 0: # printing output. You likely want to mod this with higher num epochs
        print(f"Epoch {epoch:02d} | train_loss={train_loss:.6f} | val_loss={val_loss:.6f}")


### We now evaluate the model

Let's plot our loss curve and make predictions on the test set.


In [ ]:
model.eval()
all_preds = []
true = []
with torch.no_grad():
    for xt, yt in test_loader:
        xt = xt.to(device)
        preds = # complete me
        all_preds.append(preds.cpu())   # move back to CPU for numpy/plotting
        true.append(yt.numpy())


# plot loss curve
plot_learning_curve(history)
# visualize predictions on validation data
val_preds = torch.cat(all_preds).numpy()
true = np.concatenate(true)
nbins = 50
plt.hist(true,bins=70,alpha=0.5)
plt.hist(val_preds,bins=70)
plt.show()
print(len(true))
print(len(val_preds))


## Tuning

I did *not* start you out with ideal hyperparemeters and architecture. You should now tune your model to be as good as possible, based on information from your loss curve.

Some tuning to consider to get the best possible model:

1. Would scaling your data help your model?
2. Might including charge help your model? Why/why not?
2. Might more training data help?
2. Was ReLU the best activation function for this network? Other options for a hidden layer include `Sigmoid`, `tanh`, or `ReLU` variants.
3. Perhaps we did not have enough model parameters to accurately represent the mapping. You can try to increase the number of nodes in the hidden layer and/or add mor hidden layers
3.  Another hyperparameter to adjust is batch size, which is the number of training examples used to calculate the gradient on each step. While you may initially think that a higher batch size leads to faster or more accurate training, in practice this is not true. The "noise" that arises from using less training examples at each iteration can actually help find the global minimum of the loss function. (See here for more info: https://arxiv.org/pdf/1609.04836.pdf)
4. Another hyperparameter to tune is the *learning rate*.

 - If the learning rate is too high, we are taking too large of a step in the gradient descent at each iteration and will miss narrow minima in the loss function.
 - If the learning rate is too small, then we are not traveling far enough in each iteration and we will take far too long to reach a minimum.

  Perhaps the learning rate is too high and the network can't fine tune.
5. If you see evidence of *overfitting*, meaning the validation loss begins to climb as the training loss decreases, you can try adding [dropout layers](https://docs.pytorch.org/docs/stable/generated/torch.nn.Dropout.html) or [batch normalization](https://docs.pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html).
6. There are a suite of [gradient descent-based optimizers](https://docs.pytorch.org/docs/stable/optim.html) to try



Even after tuning, you should find (expecially if you predict predictions vs ground truth), that your model isn't great. This is for more fundamental reasons on the learning problem that we set up. *What are they?*